In [33]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import time
import re

In [34]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0"}

In [35]:
base_url = "https://hilalprp.com.om/"
start_url = base_url + "/properties-search/page/{}/?status=for-rent"
max_page = 20

In [36]:
properties_data = {
        "Property Title": [],
        "Location": [],
        "Bedrooms": [],
        "Price": [],
        "Size": [],
        "Listing Type": []
}

In [40]:
for page in range(1, max_page + 1):
    page_url = current_url.format(page)
    print(f"Scraping page {page}...")
    try:
        response = requests.get(page_url, headers=headers)
        response.raise_for_status()
    except Exception as e:
        print(f"Failed to fetch page {page}: {e}")
        break

    soup = BeautifulSoup(response.text, "html.parser")
    cards = soup.find_all("article", class_="rh_list_card")
    if not cards:
        print("No more listings found.")
        break

    for card in cards:
        # Property Title
        title_tag = card.find("h3")
        title = title_tag.text.strip() if title_tag else None

        # Price
        price_tag = card.find("p", class_="price")
        price = price_tag.text.strip().replace("OMR", "").replace(",", "").strip() if price_tag else None

        # Listing Type
        properties_data["Listing Type"].append("For Rent")

        # Meta info
        bedrooms = None
        size = None
        meta_wrap = card.find_all("div", class_="rh_prop_card__meta")
        for item in meta_wrap:
            label = item.find("span", class_="rh_meta_titles")
            value = item.find("span", class_="figure")
            if label and value:
                label_text = label.text.strip().lower()
                val_text = value.text.strip()
                if "bedroom" in label_text:
                    bedrooms = val_text
                elif "area" in label_text or "size" in label_text or "sqmt" in label_text:
                    size = val_text

        # Detail page for location
        location = None
        detail_link_tag = card.find("a", href=True)
        if detail_link_tag:
            detail_url = detail_link_tag['href']
            try:
                detail_resp = requests.get(detail_url, headers=headers)
                detail_resp.raise_for_status()
                detail_soup = BeautifulSoup(detail_resp.content, 'html.parser')
                location_tag = detail_soup.find("a", href=lambda x: x and "/property-city/" in x)
                if location_tag:
                    location = location_tag.text.strip()
            except Exception as e:
                print(f"Error fetching detail page: {e}")

        # Append all data
        properties_data["Property Title"].append(title)
        properties_data["Location"].append(location)
        properties_data["Bedrooms"].append(bedrooms)
        properties_data["Price"].append(price)
        properties_data["Size"].append(size)
        

    time.sleep(1)

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
No more listings found.


In [41]:
df = pd.DataFrame(properties_data)
df

,Property Title,Location,Bedrooms,Price,Size,Listing Type
0,4-BEDROOM VILLA,Azaiba,4,500,None,For Rent
1,STUDIO FLAT,Bausher,1,270,None,For Rent
2,COMMERCIAL SHOP,Al Mawaleh,None,350,65,For Rent
3,2-BEDROOM APARTMENT,Al Ansab,2,300,None,For Rent
4,2-BEDROOM APARTMENT,Shatti Al Qurum,2,500,None,For Rent
...,...,...,...,...,...,...
182,2 BEDROOM APARTMENT IN (SEEB),Seeb,2,350,170,For Rent
183,4 BEDROOM RENOVATED DETACHED VILLA,Azaiba,4,-950,+-340,For Rent
184,6 BEDROOM DETACHED VILLA IN (SHATTI AL QURUM),Shatti Al Qurum,6,-950,+-350,For Rent
185,2 BEDROOM APARTMENT IN (BOSHER),Bausher,2,475,None,For Rent


In [42]:
df.to_csv("Hilal.csv", index = False)